In [ ]:
!nvcc --version
!nvidia-smi
%load_ext autoreload
%autoreload 2


import sys
sys.path.insert(1, '/home/kcutler/kevin_utils/')
from kevin import rescale, ncolorlabel, endpoints, getskel

import numpy as np
# import cupy as np
import time, os, sys
from urllib.parse import urlparse
from urllib.parse import urlparse
from cellpose_omni import models

use_GPU = models.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

In [ ]:
import time, os, sys
import skimage.io
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
from cellpose_omni import utils

In [ ]:
from cellpose_omni import io
from tifffile import imread
import skimage.io


In [ ]:
# basedir = '/home/kcutler/DataDrive/images_cyto_oct_server/test/'
basedir = '/home/kcutler/DataDrive/images_cyto_oct_server/train/'
mask_filter = '_masks'
img_filter = ''
img_names = io.get_image_files(basedir,mask_filter,img_filter)
mask_names,flow_names = io.get_label_files(img_names, mask_filter,img_filter)
imgs = [skimage.io.imread(f) for f in img_names]
masks = [skimage.io.imread(f) for f in mask_names]
diameters = np.array([utils.diameters(m)[0] for m in masks]) 
# print(files)
# basedir = '/home/kcutler/DataDrive/final_train/'
# mask_filter = '_masks'
# img_names = io.get_image_files(basedir,mask_filter)
# mask_names,flow_names = io.get_label_files(img_names, mask_filter)
# imgs = [imread(f) for f in img_names]
# masks = [imread(f) for f in mask_names]
# flows = [imread(f) for f in flow_names]

In [ ]:
cnt = 0;
r = [0]*len(masks)
for i,mask in enumerate(masks):
    m = utils.format_labels(mask)
    lbls = np.unique(m)
    cnt += len(lbls[lbls>0])
    fg = np.count_nonzero(m>0)
    bg = np.count_nonzero(m==0)
    r[i] = fg/(fg+bg)
    

In [ ]:
ind = np.argmin(r)
plt.imshow(rescale(imgs[ind]))
len(imgs)

In [ ]:
from cellpose_omni import models

skel=False
if skel:
    modeldir = '/home/kcutler/DataDrive/images_cyto_oct_server/train/models/cellpose_residual_on_style_on_concatenation_off_train_2021_07_28_22_55_30.617388_epoch_3999'
    nclasses = 4
    chan = [2,1]
else: 
    modeldir = 'cyto2'
    chan = [0,0]
    
diameters = np.array([utils.diameters(m,skel=skel)[0] for m in masks]) 

model = models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir, net_avg=False, nclasses=nclasses,skel=skel)
# model = models.CellposeModel(net_avg=False)

sizemodel = models.SizeModel(device=model.device, cp_model=model)

In [ ]:
# inds = range(10)
inds = range(len(imgs))
print(inds)
params = sizemodel.train([imgs[i] for i in inds],[masks[i] for i in inds],channels=chan,n_epochs=10, batch_size=8,skel=skel) #NEEDS CHANNELS ARGUMENT???

In [ ]:
inds = range(5)
print(sizemodel.diam_mean)
diam, diam_style = sizemodel.eval([imgs[i] for i in inds],channels=[0,0],skel=skel)

In [ ]:
d = np.array([diameters[i] for i in inds])
error_ds = (diam_style-d)/d
error_d = (diam-d)/d
print('mean diam_style fractional error is', np.mean(error_ds**2)**.5)
print('mean diam fractional error is', np.mean(error_d**2)**.5)

In [ ]:
np.divide(diam,[diameters[i] for i in inds])

In [ ]:
from cellpose_omni import transforms
# inds = range(8)
img,lbl,sc = transforms.random_rotate_and_resize([imgs[k][:,:,np.newaxis] for k in inds],[flows[k] for k in inds],xy=(220,220))
sizemodel.cp.network(img)

In [ ]:
sizemodel.cp.network(img.swapaxes(-1,1))

In [ ]:
# [imgs[k][np.newaxis,:,:] for k in inds]
[flows[k] for k in inds]

In [ ]:
plt.imshow(imgs[9][0])

In [ ]:
diameters = np.array([utils.diameters(m)[0] for m in masks]) 

In [ ]:
diameters

In [ ]:
model.diameters

In [ ]:
len(masks)